In [36]:
import os
import zipfile

# Path to the zip file
zip_file_path = r'C:\Users\dpamu\Downloads\daily_and_sports_activities.zip'
extract_to_path = r'C:\Users\dpamu\Downloads\daily_and_sports_activities'

# Step 1: Extract the zip file
if not os.path.exists(extract_to_path):  # Ensure the directory doesn't already exist
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)
        print(f"Extracted zip file to: {extract_to_path}")

# Step 2: List the contents of the extracted directory
print("Contents of extracted directory:", os.listdir(extract_to_path))


Contents of extracted directory: ['data']


In [26]:
#import os

# Update this path with the actual path to your extracted dataset
#base_path = r'C:\Users\dpamu\Downloads\daily_and_sports_activities\data'

# Ensure the base path exists
#if not os.path.exists(base_path):
#    raise FileNotFoundError(f"The specified path does not exist: {base_path}")

# Initialize an empty list to hold all the flattened data
#all_data = []

# Loop through activity folders (a01 to a19)
#for activity in os.listdir(base_path):
#    activity_path = os.path.join(base_path, activity)
#    if os.path.isdir(activity_path): 
#        # Loop through subject folders (p1 to p8)
#        for subject in os.listdir(activity_path):
#            subject_path = os.path.join(activity_path, subject)
#            if os.path.isdir(subject_path): 
#                # Loop through segment files (s01.txt to s60.txt)
#                for segment_file in os.listdir(subject_path):
#                    segment_path = os.path.join(subject_path, segment_file)
#                    #if segment_file.endswith(".txt"): 
#                       # print(f"Processing file: {segment_path}")


In [37]:
import os
import pandas as pd
import numpy as np

# Path to the subject folder containing segment files
subject_folder = r'C:\Users\dpamu\Downloads\daily_and_sports_activities\data\a01\p1'


# List to store all flattened rows
all_flattened_data = []

# Loop through all files in the folder
for file_name in os.listdir(subject_folder):
    if file_name.endswith(".txt"):  # Process only .txt files
        file_path = os.path.join(subject_folder, file_name)
        
        # Load and flatten the file
        data = pd.read_csv(file_path, header=None)
        flattened_data = data.values.flatten()
        all_flattened_data.append(flattened_data)

# Combine all flattened rows into a single DataFrame
combined_df = pd.DataFrame(all_flattened_data)

# Save the combined DataFrame to a CSV file
combined_df.to_csv('flattened_dataset.csv', index=False, header=False)

print(f"All files processed and combined. Shape: {combined_df.shape}")


All files processed and combined. Shape: (60, 5625)


In [38]:
combined_df.head()

,0,1,2,3,4,5,6,7,8,9,...,5615,5616,5617,5618,5619,5620,5621,5622,5623,5624
0,8.1305,1.0349,5.4217,-0.009461,0.001915,-0.003424,-0.78712,-0.069654,0.15730,0.70097,...,-0.036874,-2.8154,-9.0600,2.6025,-0.003904,-0.006729,-0.009789,0.73897,0.30275,-0.056262
1,7.9665,1.1684,5.6755,-0.005730,0.026995,-0.009029,-0.79062,-0.071635,0.13429,0.62906,...,-0.038551,-2.8233,-9.0757,2.6337,-0.006769,-0.006575,-0.004326,0.74027,0.30192,-0.057155
2,7.8917,1.1390,5.6980,0.014180,0.028722,-0.009079,-0.79531,-0.069460,0.12447,0.60430,...,-0.040145,-2.8091,-9.0846,2.6295,-0.000714,-0.002681,0.004770,0.74072,0.30101,-0.057301
3,7.9366,1.1536,5.6318,0.003242,0.029965,0.009111,-0.79292,-0.070358,0.13194,0.59417,...,-0.041109,-2.8844,-9.0849,2.6298,-0.010604,-0.002827,-0.004194,0.74150,0.30305,-0.055743
4,7.8913,1.1972,5.9082,-0.044333,-0.067467,-0.004235,-0.79592,-0.073174,0.12086,0.56915,...,-0.039495,-2.8249,-9.1083,2.6322,0.013583,0.013670,0.007613,0.74007,0.30324,-0.055548


### Since the each sec segment sampled at 25Hz we have 125 samples/senssor in 5 sec. This is producing 5625 features. The given instances in 9120 which is not enough to support the given number of features. To mitigate the risk of overfitting, we are reducing the number of features by utilizing only a one-second segment.

In [39]:
import os
import pandas as pd

# Path to the base folder containing all activity folders
base_path = r'C:\Users\dpamu\Downloads\daily_and_sports_activities\data'

#Buckets for the labels
activity_mapping = {
    "a01": "Sitting",
    "a02": "Standing",
    "a03": "Lying on Back",
    "a04": "Lying on Right Side",
    "a05": "Ascending Stairs",
    "a06": "Descending Stairs",
    "a07": "Standing in Elevator",
    "a08": "Moving in Elevator",
    "a09": "Walking in Parking Lot",
    "a10": "Walking on Treadmill (Flat)",
    "a11": "Walking on Treadmill (Inclined)",
    "a12": "Running on Treadmill",
    "a13": "Exercising on Stepper",
    "a14": "Exercising on Cross Trainer",
    "a15": "Cycling (Horizontal)",
    "a16": "Cycling (Vertical)",
    "a17": "Rowing",
    "a18": "Jumping",
    "a19": "Playing Basketball"
}


# List to store all data
all_flattened_data = []
all_activities = []
all_persons = []

#Looping through activities,person, sensors values
for activity in os.listdir(base_path):
    activity_path = os.path.join(base_path, activity)
    if os.path.isdir(activity_path):  
        # Loop through person folders (p1 to p8)
        for person in os.listdir(activity_path):
            person_path = os.path.join(activity_path, person)
            if os.path.isdir(person_path):  
                # Loop through segment files (s01.txt to s60.txt)
                for file_name in os.listdir(person_path):
                    if file_name.endswith(".txt"):  # Process only .txt files
                        file_path = os.path.join(person_path, file_name)
                        
                        # Load/flatten the file
                        data = pd.read_csv(file_path, header=None)
                        data_1sec = data.iloc[:25, :]  # Keeping only 1-second (25 samples)
                        flattened_data = data_1sec.values.flatten()
                        all_flattened_data.append(flattened_data)

                        # Add activity with lables and person info

                        all_activities.append(activity_mapping.get(activity, "Unknown"))
                        all_persons.append(person)



# Combinng all to a single DataFrame
combined_df = pd.DataFrame(all_flattened_data)

# Add activity and person columns
combined_df.insert(0, 'Activity', all_activities)
combined_df.insert(1, 'Person', all_persons)


print(f"Final dataframe with reduced features , activity and person info. Shape: {combined_df.shape}")


# Exclude non-numeric columns for correlation calculation
#numeric_data = combined_df.drop(columns=['Activity', 'Person'])

# Compute the correlation matrix for numeric columns
#correlation_matrix = numeric_data.corr()

#print("Correlation matrix computed and saved.")

#Since all data comprises of sensor measurements it all fields would show correlation
#plt.figure(figsize=(12, 10))
#sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False, fmt=".2f", cbar=True)
#plt.title("Feature Correlation Heatmap")
#plt.show()


Final dataframe with reduced features , activity and person info. Shape: (9120, 1127)


In [40]:
combined_df.head()

,Activity,Person,0,1,2,3,4,5,6,7,...,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124
0,Sitting,p1,8.1305,1.0349,5.4217,-0.009461,0.001915,-0.003424,-0.78712,-0.069654,...,-0.037685,-2.8225,-9.0743,2.6232,0.001613,0.011201,-0.007894,0.74027,0.30125,-0.056569
1,Sitting,p1,7.9665,1.1684,5.6755,-0.005730,0.026995,-0.009029,-0.79062,-0.071635,...,-0.036265,-2.7930,-9.1125,2.6326,-0.000412,0.020459,0.004848,0.73920,0.30238,-0.058635
2,Sitting,p1,7.8917,1.1390,5.6980,0.014180,0.028722,-0.009079,-0.79531,-0.069460,...,-0.038679,-2.8087,-9.0839,2.6355,-0.004004,0.005884,-0.011469,0.73969,0.30330,-0.057885
3,Sitting,p1,7.9366,1.1536,5.6318,0.003242,0.029965,0.009111,-0.79292,-0.070358,...,-0.039223,-2.8390,-9.0621,2.6365,-0.009599,0.005176,-0.004187,0.74128,0.30088,-0.056882
4,Sitting,p1,7.8913,1.1972,5.9082,-0.044333,-0.067467,-0.004235,-0.79592,-0.073174,...,-0.039495,-2.8245,-9.0777,2.6159,0.000228,-0.000993,-0.003331,0.74114,0.30291,-0.055221


In [41]:
#Exploring data types in the data frame
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9120 entries, 0 to 9119
Columns: 1127 entries, Activity to 1124
dtypes: float64(1125), object(2)
memory usage: 78.4+ MB


In [42]:
#check for null data
import numpy as np
combined_df.isnull().sum()

Activity    0
Person      0
0           0
1           0
2           0
           ..
1120        0
1121        0
1122        0
1123        0
1124        0
Length: 1127, dtype: int64

In [43]:
combined_df.describe()


,0,1,2,3,4,5,6,7,8,9,...,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124
count,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,...,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000
mean,7.879943,-0.807308,2.823025,-0.005078,0.022446,-0.002455,-0.601650,0.060835,-0.272478,4.302443,...,0.069614,-7.422868,-3.165476,-0.497145,-0.022566,0.025041,-0.013709,0.461421,0.294218,0.064795
std,5.706403,2.653094,3.667117,0.791649,0.687893,0.334184,0.352953,0.342807,0.374478,5.931536,...,0.364298,5.834603,6.169920,3.389488,0.880451,0.485740,1.167599,0.379462,0.383986,0.359813
min,-13.416000,-26.959000,-18.764000,-11.700000,-10.152000,-4.491300,-1.345900,-1.001400,-1.029900,-19.833000,...,-0.759700,-93.324000,-86.960000,-24.292000,-9.314100,-3.307700,-6.024800,-0.842210,-0.893920,-0.912800
25%,6.981500,-1.503850,0.958283,-0.174487,-0.102540,-0.102270,-0.824767,-0.185420,-0.604215,0.125690,...,-0.215045,-9.816150,-7.914425,-2.179175,-0.315578,-0.125010,-0.401803,0.296325,0.028546,-0.174642
50%,8.869300,-0.400360,2.750500,0.000860,0.018424,-0.002962,-0.690060,0.024050,-0.311025,4.230400,...,0.068505,-8.653450,-2.322350,-0.762290,-0.004837,0.001032,-0.000904,0.511435,0.350645,0.084097
75%,9.713800,0.426432,4.428400,0.160235,0.134960,0.089188,-0.500007,0.285342,0.043585,8.572225,...,0.336905,-2.777000,0.219138,0.415428,0.183945,0.143110,0.590180,0.694597,0.581883,0.299375
max,78.967000,23.190000,67.973000,9.393200,6.852600,7.706400,0.879190,1.029900,0.900980,62.497000,...,1.054500,8.724100,54.195000,19.701000,16.533000,4.995800,4.986600,1.371200,1.120600,1.020600


### Normalizing data by using StandardScaler after splitting the data
### In order  to prevent information leakage data is being split into training and test datasets first and then applying standard scaler
### Dropping the Person column as it is not a relevant feature to train the model on.
### X being the sensors measurements Y is the target variable - Activity

In [44]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


X = combined_df.drop(columns=['Activity', 'Person'])
y = combined_df['Activity']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Applying StandardScaler
scaler = StandardScaler()

# Fitting the scaler on the training set
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test set using the same scaler
X_test_scaled = scaler.transform(X_test)


print(f"Training data shape: {X_train_scaled.shape}")
print(f"Test data shape: {X_test_scaled.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test labels shape: {y_test.shape}")


Training data shape: (7296, 1125)
Test data shape: (1824, 1125)
Training labels shape: (7296,)
Test labels shape: (1824,)


In [45]:
print("X train scaled features\n",X_train_scaled[:5])
print("X test scaled features\n",X_test_scaled[:5])
print("Train activities\n",y_train[:5])
print("Test activities\n", y_test[:5])

X train scaled features
 [[ 0.28867335 -0.32087923 -0.59295888 ...  0.16798973  1.35911624
  -0.35211945]
 [ 0.21509887  1.27669702 -1.1770791  ...  0.07355653  0.78693165
  -0.7057678 ]
 [-0.98215222 -0.51254477 -1.4170887  ...  1.92231672  0.27437352
  -0.65047309]
 [ 0.29429632  0.48727788 -1.64662734 ...  0.15081289  0.66363937
  -0.68315227]
 [ 0.99374746  0.18006787  0.2349646  ... -0.57705933  1.19055685
   0.30128703]]
X test scaled features
 [[ 0.25786162  0.21465639  0.0784211  ...  0.43679557 -0.0773652
   1.24594179]
 [-0.01813964  0.21888221 -1.07039598 ...  0.88163121  0.49719089
   0.0671416 ]
 [-0.07484523  0.25283912  0.42645695 ... -0.46013577  0.68975201
  -0.35723697]
 [ 0.1043141   0.93584968  0.23304584 ...  0.31184912 -1.18850721
   1.03725957]
 [ 0.06927192  0.57142574 -0.34461089 ...  0.15186507  1.21187435
   0.22395739]]
Train activities
 5165    Walking on Treadmill (Inclined)
2090                   Ascending Stairs
5408               Running on Treadmill
76